In [1787]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split

In [1788]:
#importing data
os.chdir("C:/Users/Joey/Desktop/Pima")
os.listdir()
data=pd.read_csv("pima-indians-diabetes.csv")
#Storing 10 runs
averageAccuracy=[]

In [2022]:
data

Number of pregnancies  Glucose concentration  Blood Pressure  \
0                        6                    148              72   
1                        1                     85              66   
2                        8                    183              64   
3                        1                     89              66   
4                        0                    137              40   
..                     ...                    ...             ...   
763                     10                    101              76   
764                      2                    122              70   
765                      5                    121              72   
766                      1                    126              60   
767                      1                     93              70   

     Skin thickness  Insulin  Body mass index  Diabetes pedigree function  \
0                35        0             33.6                       0.627   
1                29        0             26.6                       0.351   
2                 0        0             23.3                       0.672   
3                23       94             28.1                       0.167   
4                35      168             43.1                       2.288   
..              ...      ...              ...                         ...   
763              48      180             32.9                       0.171   
764              27        0             36.8                       0.340   
765              23      112             26.2                       0.245   
766               0        0             30.1                       0.349   
767              31        0             30.4                       0.315   

     Age  Outcome  
0     50        1  
1     31        0  
2     32        1  
3     21        0  
4     33        1  
..   ...      ...  
763   63        0  
764   27        0  
765   30        0  
766   47        1  
767   23        0  

[768 rows x 9 columns]

In [2023]:
#splitting the data
X_train,X_test=train_test_split(data,test_size=0.5)

In [2024]:
class1=X_train[X_train["Outcome"]==0]
class2=X_train[X_train["Outcome"]==1]

In [2025]:
def mean(X):
    X_mean=np.mean(X,axis=0)
    return X_mean

In [2026]:
MeanClass1=mean(class1.iloc[:,:8])
MeanClass2=mean(class2.iloc[:,:8])
trMean=mean(X_train.iloc[:,:8])

In [2027]:
MeanClass1=MeanClass1.to_numpy().reshape(MeanClass1.shape[0],1)
MeanClass2=MeanClass2.to_numpy().reshape(MeanClass1.shape[0],1)
trMean=trMean.to_numpy().reshape(trMean.shape[0],1)

In [2028]:
S1=(class1.shape[0]-1)*np.dot((class1.iloc[:,:8]-MeanClass1.T).T,(class1.iloc[:,:8]-MeanClass1.T))
S2=(class2.shape[0]-1)*np.dot((class2.iloc[:,:8]-MeanClass2.T).T,(class2.iloc[:,:8]-MeanClass2.T))
Sw=S1+S2

In [2029]:
Sw

array([[ 7.30228222e+05,  5.08753289e+05,  5.81007471e+05,
        -5.48495004e+05, -3.10028442e+06, -6.41858775e+04,
        -7.36895955e+03,  1.39028744e+06],
       [ 5.08753289e+05,  6.61498655e+07,  8.50584749e+06,
         2.36162112e+06,  9.96458419e+07,  2.87530592e+06,
         3.89726223e+04,  3.78282952e+06],
       [ 5.81007471e+05,  8.50584749e+06,  3.48619805e+07,
         6.55112381e+06,  2.63387265e+07,  4.75674082e+06,
         2.05739585e+04,  3.64846731e+06],
       [-5.48495004e+05,  2.36162112e+06,  6.55112381e+06,
         1.83670180e+07,  6.29232470e+07,  3.98186510e+06,
         7.91794025e+04, -2.45555015e+06],
       [-3.10028442e+06,  9.96458419e+07,  2.63387265e+07,
         6.29232470e+07,  1.15332953e+09,  1.71461092e+07,
         7.12153796e+05, -5.76344935e+06],
       [-6.41858775e+04,  2.87530592e+06,  4.75674082e+06,
         3.98186510e+06,  1.71461092e+07,  4.74232785e+06,
         2.38713153e+04,  1.21413409e+05],
       [-7.36895955e+03,  3.897262

In [2030]:
trMean.shape

(8, 1)

In [2031]:
Sb1=(class1.shape[0]-1)*np.dot((trMean-MeanClass1.T),(trMean-MeanClass1.T))
Sb2=(class2.shape[0]-1)*np.dot((trMean-MeanClass2.T),(trMean-MeanClass2.T))
Sb=Sb1+Sb2

In [2032]:
inv_Sw=np.linalg.inv(Sw)

In [2033]:
MeanClass1.shape

(8, 1)

In [2034]:
optimal_direction=np.dot(inv_Sw,MeanClass1-MeanClass2)
optimal_direction

array([[-1.74487731e-06],
       [-4.60061551e-07],
       [ 2.24025400e-07],
       [-1.32115311e-07],
       [ 4.48124054e-08],
       [-1.15276161e-06],
       [-1.79950359e-05],
       [-5.84431169e-07]])

In [2035]:
Class1_LDA=class1.iloc[:,:8]*optimal_direction.T
Class2_LDA=class2.iloc[:,:8]*optimal_direction.T


In [2036]:
X_trainA=Class1_LDA
X_trainB=Class2_LDA

In [2037]:
prior_probability_A=X_trainA.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])
prior_probability_B=X_trainB.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])

In [2038]:
#function which takes "mean" and "covariance" as the parameters and returns the likelihood of the Feature Vector
def likelihood(x,mean,cov):
    #inverse of the covariance matrix
    inv=np.linalg.inv(cov)
    t1=1/(np.sqrt(((2*np.pi)**3)*np.linalg.det(cov)))
    t2=np.exp(-0.5*np.dot(np.dot((x-mean).T,inv),(x-mean)))
    t=t1*t2
    return t

In [2039]:
#calculating the covariance matrix for both classes(A and B)
covarianceA=np.cov(X_trainA.iloc[:,:].T)
covarianceB=np.cov(X_trainB.iloc[:,:].T)

In [2040]:
MeanVecA=np.mean(X_trainA,axis=0)
MeanVecB=np.mean(X_trainB,axis=0)

In [2041]:
MeanVecA=MeanVecA.to_numpy().reshape(1,MeanVecA.shape[0])
MeanVecB=MeanVecB.to_numpy().reshape(1,MeanVecB.shape[0])


In [2042]:
X_test


Number of pregnancies  Glucose concentration  Blood Pressure  \
764                      2                    122              70   
107                      4                    144              58   
387                      8                    105             100   
7                       10                    115               0   
98                       6                     93              50   
..                     ...                    ...             ...   
178                      5                    143              78   
171                      6                    134              70   
288                      4                     96              56   
759                      6                    190              92   
466                      0                     74              52   

     Skin thickness  Insulin  Body mass index  Diabetes pedigree function  \
764              27        0             36.8                       0.340   
107              28      140             29.5                       0.287   
387              36        0             43.3                       0.239   
7                 0        0             35.3                       0.134   
98               30       64             28.7                       0.356   
..              ...      ...              ...                         ...   
178               0        0             45.0                       0.190   
171              23      130             35.4                       0.542   
288              17       49             20.8                       0.340   
759               0        0             35.5                       0.278   
466              10       36             27.8                       0.269   

     Age  Outcome  
764   27        0  
107   37        0  
387   45        1  
7     29        0  
98    23        0  
..   ...      ...  
178   47        0  
171   29        1  
288   26        0  
759   66        1  
466   22        0  

[384 rows x 9 columns]

In [2043]:
X_testA=X_test.iloc[:,:8]
def accuracy(X_test,Optimal_direction,MeanVecA,MeanVecB,covarianceA,covarianceB,prior_probability_A,prior_probability_B):
    a=X_test.iloc[:,:8]*Optimal_direction.T
    count=0
    for i in range(X_test.shape[0]):
        p=a.iloc[i,:].to_numpy().reshape(a.shape[1],1)
        posA=likelihood(p,MeanVecA.T,covarianceA)*prior_probability_A
        posB=likelihood(p,MeanVecB.T,covarianceB)*prior_probability_B
        if(posA<posB):
            if(X_test.iloc[i,8]==1):
                count+=1
        else:
            if(X_test.iloc[i,8]==0):
                count+=1
    classificationAccuracy=count/X_test.shape[0]
    return classificationAccuracy

In [2044]:
avgAccuracy = accuracy(X_test,optimal_direction,MeanVecA,MeanVecB,covarianceA,covarianceB,prior_probability_A,prior_probability_B)

In [2045]:
averageAccuracy.append(avgAccuracy)


In [2046]:
averageAccuracy

[0.7317708333333334,
 0.7369791666666666,
 0.765625,
 0.734375,
 0.7161458333333334,
 0.7473958333333334,
 0.75,
 0.7708333333333334,
 0.75,
 0.734375]

In [2048]:
print(np.mean(averageAccuracy))

0.74375
